In [24]:
%matplotlib inline
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

In [25]:

def min_result(list1, n):
    temp = list1
    result = list1
    window = 1
    for iter in range(n):
        temp = np.minimum(temp[0:(np.size(temp)-window)], temp[window:])
        result = np.concatenate((result, temp))
        window = window * 2
    return result


def max_result(list1, n):
    temp = list1
    result = list1
    window = 1
    for iter in range(n):
        temp = np.maximum(temp[0:(np.size(temp)-window)], temp[window:])
        result = np.concatenate((result, temp))
        window = window * 2
    return result

# list1, list2 are the average of the first and second moments of the datapoints
def avg_sd_result(list1, list2, n):
    temp1 = list1
    temp2 = list2
    result1 = temp1
    result2 = temp2
    window = 1
    for iter in range(n):
        temp1 = (temp1[0:(np.size(temp1)-window)] + temp1[window:])/2
        temp2 = (temp2[0:(np.size(temp2)-window)] + temp2[window:])/2
        result1 = np.concatenate((result1, temp1))
        result2 = np.concatenate((result2, temp2))
        window = window * 2
    result2 = np.sqrt(result2 - np.square(result1))
    return np.concatenate((result1, result2))

def min_max_avg_sd(stats, n):
    result_min = min_result(stats[:, 0], n)
    result_max = max_result(stats[:, 1], n)
    result_avg_sd = avg_sd_result(stats[:, 2], np.square(stats[:, 2]) + np.square(stats[:, 3]), n)
    # result_avg_sd = avg_sd_result(stats[:, 2], np.square(stats[:, 2]) + np.square(stats[:, 3]) * (1-1/np.stats[:, 4]), n)
    return np.concatenate((result_min, result_max, result_avg_sd))


In [13]:
def ParseBoiler(filename):
    
    dfOrig = pd.read_csv('/dfs/scratch0/bosch/BG-Data_Part11/2029718103746674690.csv',
                     parse_dates=[['Date','Time']], delimiter='\t')
    dfOrig2 = dfOrig.drop([0]) #TODO - remove this?
    dfOrig2['Date_Time'] = pd.to_datetime(dfOrig2['Date_Time'], format='%d.%m.%Y %H:%M:%S', errors='coerce')
    
    #Only select the columns we care about
    dfSubset = dfOrig2[['Date_Time', 'Operating_status:_Error_Locking', 'Operating_status:_Error_Blocking',
                   'Actual_Power', 'Number_of_burner_starts', 'Operating_status:_Central_heating_active',
                  'Operating_status:_Hot_water_active', 'Operating_status:_Flame', 'Relay_status:_Gasvalve',
                  'Relay_status:_Fan', 'Relay_status:_Ignition', 'Relay_status:_internal_3-way-valve',
                  'Relay_status:_HW_circulation_pump', 'Supply_temperature_(primary_flow_temperature)',
                  'Maximum_supply_(primary_flow)_temperature', 'Hot_water_temperature_setpoint', 
                  'Hot_water_outlet_temperature', 'Actual_flow_rate_turbine', 'Fan_speed']]
    
    #Fill in missing entries
    df = dfSubset.fillna(method = 'ffill')
    
    print "Successfully loaded", dfOrig
    
    Positives = df.loc[(df['Operating_status:_Error_Locking'] == 1) 
                       & (df['Operating_status:_Error_Locking'].shift(1) == 0)]
    
    # I added several lines here
    Positive_result = np.zeros((np.size(Positives, 0), 4388 * 16)) 
    outer_iter = 0
    # End of the lines added
    
    for index, row in Positives.iterrows():
        #For each Positive example...
        eventTime = row['Date_Time']
        startTime = eventTime - pd.Timedelta('7 days')
        dfPos = df[(df['Date_Time'] >= startTime) & (df['Date_Time'] < eventTime)]
        
        print dfPos.shape
        
        sensorList = ['Actual_Power', 'Number_of_burner_starts', 'Operating_status:_Central_heating_active',
                  'Operating_status:_Hot_water_active', 'Operating_status:_Flame', 'Relay_status:_Gasvalve',
                  'Relay_status:_Fan', 'Relay_status:_Ignition', 'Relay_status:_internal_3-way-valve',
                  'Relay_status:_HW_circulation_pump', 'Supply_temperature_(primary_flow_temperature)',
                  'Maximum_supply_(primary_flow)_temperature', 'Hot_water_temperature_setpoint', 
                  'Hot_water_outlet_temperature', 'Actual_flow_rate_turbine', 'Fan_speed']
        
        # I added several lines here
        inner_iter = 0
        # End of the lines added
        
        for sensor in sensorList:
            print "Getting data for", sensor
            
            rawData = np.zeros((168,5))
            for j in range(168):
                lastTime = startTime + j*pd.Timedelta('1 hour')
                currTime = startTime + (j+1)*pd.Timedelta('1 hour')
                
                dfHourly = dfPos[(dfPos['Date_Time'] >= lastTime) & (dfPos['Date_Time'] < currTime)]

                description = dfHourly[sensor].describe()
                rawData[j,0] = description['min']
                rawData[j,1] = description['max']
                rawData[j,2] = description['mean']
                rawData[j,3] = description['std']
                rawData[j,4] = description['count']

                # I added several lines here
            Positive_result[outer_iter, (inner_iter * 4388):(inner_iter+1)*4388] = min_max_avg_sd(rawData, 7)    
            inner_iter = inner_iter + 1
            
        outer_iter = outer_iter + 1
        # End of the lines added    

In [26]:
#Parse ALL Boilers - TODO - unzip other folders too!
for filename in os.listdir('/dfs/scratch0/bosch/BG-Data_Part11/'):
     ParseBoiler(filename)

NameError: name 'os' is not defined

In [ ]:
#Parse just one boiler (for testing...)
ParseBoiler('/dfs/scratch0/bosch/BG-Data_Part11/2029718103746674690.csv')

In [7]:
# Here below the codes are used for testing

In [27]:
filename = '/dfs/scratch0/bosch/BG-Data_Part11/2029718103746674690.csv'

In [28]:
dfOrig = pd.read_csv('/dfs/scratch0/bosch/BG-Data_Part11/2029718103746674690.csv',parse_dates=[['Date','Time']], delimiter='\t')
dfOrig2 = dfOrig.drop([0]) #TODO - remove this?
dfOrig2['Date_Time'] = pd.to_datetime(dfOrig2['Date_Time'], format='%d.%m.%Y %H:%M:%S', errors='coerce')

In [33]:
dfSubset = dfOrig2[['Date_Time', 'Operating_status:_Error_Locking', 'Operating_status:_Error_Blocking',
                   'Actual_Power', 'Number_of_burner_starts', 'Operating_status:_Central_heating_active',
                  'Operating_status:_Hot_water_active', 'Operating_status:_Flame', 'Relay_status:_Gasvalve',
                  'Relay_status:_Fan', 'Relay_status:_Ignition', 'Relay_status:_internal_3-way-valve',
                  'Relay_status:_HW_circulation_pump', 'Supply_temperature_(primary_flow_temperature)',
                  'Maximum_supply_(primary_flow)_temperature', 'Hot_water_temperature_setpoint', 
                  'Hot_water_outlet_temperature', 'Actual_flow_rate_turbine', 'Fan_speed']]
    
#Fill in missing entries
df = dfSubset.fillna(method = 'ffill')
    
print "Successfully loaded", dfOrig

Successfully loaded                    Date_Time  System_time  ECU-Type  ECU_SW-Version  \
0        16.04.  44 18:20:03          NaN        95            3081   
1        16.04.2014 18:13:32          NaN       NaN             NaN   
2        16.04.2014 18:13:33          NaN       NaN             NaN   
3        16.04.2014 18:13:33      3733920       NaN             NaN   
4        16.04.2014 18:13:36          NaN       NaN             NaN   
5        16.04.2014 18:13:37          NaN       NaN             NaN   
6        16.04.2014 18:13:37          NaN       201           33031   
7        16.04.2014 18:13:38          NaN       NaN             NaN   
8        16.04.2014 18:13:52          NaN        95            3081   
9        16.04.2014 18:13:52          NaN       NaN             NaN   
10       16.04.2014 18:13:52          NaN       NaN             NaN   
11       16.04.2014 18:13:53          NaN       NaN             NaN   
12       16.04.2014 18:13:53          NaN       NaN      

In [34]:
Positives = df.loc[(df['Operating_status:_Error_Locking'] == 1) 
                       & (df['Operating_status:_Error_Locking'].shift(1) == 0)]
    

In [36]:
Positive_result = np.zeros((np.size(Positives, 0), 4388 * 16)) 
outer_iter = 0
rawData = np.zeros((168,5))

In [55]:
  for index, row in Positives.iterrows():
        #For each Positive example...
        eventTime = row['Date_Time']
        startTime = eventTime - pd.Timedelta('7 days')
        dfPos = df[(df['Date_Time'] >= startTime) & (df['Date_Time'] < eventTime)]
        
        print dfPos.shape
        
        sensorList = ['Actual_Power', 'Number_of_burner_starts', 'Operating_status:_Central_heating_active',
                  'Operating_status:_Hot_water_active', 'Operating_status:_Flame', 'Relay_status:_Gasvalve',
                  'Relay_status:_Fan', 'Relay_status:_Ignition', 'Relay_status:_internal_3-way-valve',
                  'Relay_status:_HW_circulation_pump', 'Supply_temperature_(primary_flow_temperature)',
                  'Maximum_supply_(primary_flow)_temperature', 'Hot_water_temperature_setpoint', 
                  'Hot_water_outlet_temperature', 'Actual_flow_rate_turbine', 'Fan_speed']
        
        # I added several lines here
        inner_iter = 0
        # End of the lines added
        
        for sensor in sensorList:
            print "Getting data for", sensor
            
            for j in range(168):
                lastTime = startTime + j*pd.Timedelta('1 hour')
                currTime = startTime + (j+1)*pd.Timedelta('1 hour')
                
                dfHourly = dfPos[(dfPos['Date_Time'] >= lastTime) & (dfPos['Date_Time'] < currTime)]

                description = dfHourly[sensor].describe()
                rawData[j,0] = description['min']
                rawData[j,1] = description['max']
                rawData[j,2] = description['mean']
                rawData[j,3] = description['std']
                rawData[j,4] = description['count']

            # I added several lines here
            Positive_result[outer_iter, (inner_iter * 4388):(inner_iter+1)*4388] = min_max_avg_sd(rawData, 7)    
            inner_iter = inner_iter + 1
            
        outer_iter = outer_iter + 1
        # End of the lines added    

(115647, 19)
Getting data for Actual_Power
Getting data for Number_of_burner_starts
Getting data for Operating_status:_Central_heating_active
Getting data for Operating_status:_Hot_water_active
Getting data for Operating_status:_Flame
Getting data for Relay_status:_Gasvalve
Getting data for Relay_status:_Fan
Getting data for Relay_status:_Ignition
Getting data for Relay_status:_internal_3-way-valve
Getting data for Relay_status:_HW_circulation_pump
Getting data for Supply_temperature_(primary_flow_temperature)
Getting data for Maximum_supply_(primary_flow)_temperature
Getting data for Hot_water_temperature_setpoint
Getting data for Hot_water_outlet_temperature
Getting data for Actual_flow_rate_turbine
Getting data for Fan_speed


KeyboardInterrupt: 

In [56]:
rawData

array([[ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,

In [40]:
temp = min_max_avg_sd(rawData, 7)

In [54]:
Positive_result[0, 4388*10]

27.199999999999999

In [18]:
Positives.iterrows

<bound method DataFrame.iterrows of                   Date_Time  Operating_status:_Error_Locking  \
1860571 2014-08-08 16:09:57                                1   
1888820 2014-08-10 09:30:38                                1   
2671243 2014-10-01 16:02:57                                1   
2791898 2014-10-09 05:32:03                                1   
3316704 2014-11-13 06:31:38                                1   
3696099 2015-02-18 17:01:29                                1   
3697801 2015-02-18 20:18:27                                1   
3989781 2015-03-08 17:02:59                                1   
4818960 2015-04-25 18:10:59                                1   
4951300 2015-05-03 18:13:32                                1   
4952051 2015-05-03 18:59:35                                1   
5201866 2015-05-18 04:27:25                                1   

         Operating_status:_Error_Blocking  Actual_Power  \
1860571                                 0            32   
1888820      

<bound method DataFrame.iterrows of                   Date_Time  Operating_status:_Error_Locking  \
1860571 2014-08-08 16:09:57                                1   
1888820 2014-08-10 09:30:38                                1   
2671243 2014-10-01 16:02:57                                1   
2791898 2014-10-09 05:32:03                                1   
3316704 2014-11-13 06:31:38                                1   
3696099 2015-02-18 17:01:29                                1   
3697801 2015-02-18 20:18:27                                1   
3989781 2015-03-08 17:02:59                                1   
4818960 2015-04-25 18:10:59                                1   
4951300 2015-05-03 18:13:32                                1   
4952051 2015-05-03 18:59:35                                1   
5201866 2015-05-18 04:27:25                                1   

         Operating_status:_Error_Blocking  Actual_Power  \
1860571                                 0            32   
1888820      

In [22]:
rawData

array([[ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,  nan,   0.],
       [ nan,  nan,  nan,

In [ ]:
row = Positive